## Power analysis for proposed experiment to measure decay rate of Mtb genomes ##

*20 Dec 2024*

*Last edited: 31 July 2025*

This code simulates the proposed experiment for measuring the decay rate of Mtb genomes in mice.

Because indivdual mouse data were unavailable, we had to make assumptions about sources of uncertainty, in order to perform the simulations.

Based on Fig. 4A of Munoz-Elias *et al*., Infection and Immunity, 2005, we assumed that viable bacteria population at 4 weeks post-infection was log-normally distributed about the value $10^5$, with standard deviation equal to 0.2 log10.

In the same figure, 4A, initial average CEQs were actually lower than initial average CFUs, which corresponds to an unreasonable negative population of dead bacteria, in the DD model. To deal with this, we assumed a log-normally distributed population of dead bacteria genomes at 4 weeks post-infection. We chose the center of that distribution to correspond to CFUs/CEQs = 0.1, 0.5, or 0.9. Note that averages will differe slightly from these values. We then chose the standard deviation of the distribution to match the estimated *absolute* uncertainty in CEQs, based on the published calibration curve for qPCR (Fig. 2B of Munoz-Elias *et al*., Infection and Immunity, 2005); note that uncertainty would be drastically underestimated if we matched the uncertainty of log D to that of log Q, since D is orders of magnitude smaller than Q.

The replication rate is based on our analysis of the data in Fig. 2A, between 4 and 8 weeks post-infection ($r_3$).

We then used the estimated slope of the CFUs vs. time in Fig. 4A to estimate the death rate of Mtb in mice during treatment with INH.

In [1]:
#import necessary packages
from math import *
import numpy as np
from scipy import stats



In [2]:
#define the functions for Dependent Dynamics Model
def b(dt,r,delta,b0):
    return b0 * exp((r - delta) * dt)

def d(dt,r,delta,b0,d0,deltaD):
    rho = r - delta + deltaD
    return (delta * b0 / rho) * (exp(rho*dt) - 1) * exp(-deltaD*dt) + d0 * exp(-deltaD*dt)

def q(dt,r,delta,b0,q0,deltaD):
    d0 = q0 - b0
    if (d0 < 0.):
        d0 = 0.
    return d(dt,r,delta,b0,d0,deltaD) + b(dt,r,delta,b0)

def z(dt,r,delta,b0,d0,deltaD):
    return b(dt,r,delta,b0)/q(dt,r,delta,b0,d0,deltaD)


In [3]:
#convert from variance of log Q0 to a corresponding variance of log D0
meanLogB0 = 5 #center of log normal distribution of B0
meanB0 = pow(10,meanLogB0)  #most probable initial value of B0 (which is close to, but not exactly, the mean value)
meanZ0 = 0.9 #assumed initial value of Z = B0/Q0
sLogQ0 = 0.6 #uncertainty in log10 Q0, based on calibration curve in Munoze-Elias et al Infection and Immunity 2005 Figure 2B
varLogQ0 = sLogQ0**2 #Variance of log Q0
#mean of log10 Q0, assuming Q0 is log-normally distributed with mean value meanB0/meanZ0 (this is an approximation):
meanLogQ0 = log10(meanB0/meanZ0) - 0.5*varLogQ0                     
meanQ0 = pow(10,meanLogQ0+varLogQ0/2) #mean of Q0, assuming log normal distribution with the parametrs calculated above
varQ0 = (pow(10,varLogQ0)-1)*pow(10,2*meanLogQ0+varLogQ0) #variance of Q0
sQ0 = sqrt(varQ0) #standard deviaiton of Q0
meanD0 = meanQ0-meanB0 #average D0
sD0 = sQ0 #Choose a distribution of D0 that has the same *absolute* standard deviation as the distribution of Q0
varD0 = sD0**2 #Variance of D0
meanLogD0 = log10(meanD0**2/sqrt(meanD0**2+varD0)) #mean of corresponding log-normal distribution of D0
varLogD0 = log10(1 + varD0/(meanD0**2)) #variance of corresponding log-normal distribution of D0
sLogD0 = sqrt(varLogD0) #standard deviation of the same distribution
sLogB0 = 0.2 #assumed uncertainty in B0, based on Munoze-Elias et al Infection and Immunity 2005 Figure 2A


In [4]:
#####Functions to draw random log B and log D from normal distributions

#Function to draw a random starting CFUs (4 weeks post-infection)
def newLogB0():
    return np.random.normal(meanLogB0,sLogB0)

#Function to draw a random starting number of genoms from dead bacteria (4 weeks post-infection)
def newLogD0():
    return np.random.normal(meanLogD0,sLogD0)


In [5]:
#Function that simulates the dynamics.
#Variables:
#dt = no. of weeks between 2nd and 3rd measurements
#times = times of individual measurements; 0. is 4 weeks post-infection, 28 is 8 weeks post-infection, etc.
#nSignificant = number of simulated experiments with p < alpha (alpha is set to 0.05)
#nMice = number of mice *per group* (i.e., for each timepoint)
#alpha = threshold p-value for statistical significance
#averageF = accumulates average F statistic
#averageP = accumulates average P value
#for several of these variables, capital "T" indicates treated, as opposed to control group; not 
#relevant for this analysis, but is in the code, because we did some other analysis that were more 
#complex.

def power( nTests, alpha, nMice, weeksApart, rates ):
    dt = weeksApart * 7.
    times=[0.,28., 28. + dt]
    nSignificant = 0.
    nSignificant02 = 0.
    averageF = 0.
    averageP = 0.
    for test in range(nTests):
        rT = rates[0]
        deltaT = rates[1]
        deltaD = rates[2]
        deltaQ = deltaD
        pList = []
        bTlist = np.zeros((3,nMice)) #arrays to hold B and Q
        qTlist = np.zeros((3,nMice)) #at the different time points
        tList = [0., 28., 28. + dt]
        for p in range(len(tList)):
            for m in range(nMice):
                t = tList[p]
                b0 = pow(10.,newLogB0())
                q0 = b0 + pow(10.,newLogD0())
                #q0list.append( q0 )
                btpm = b(t,rT,deltaT,b0) #btmp = cfus (B) for a given test (t) for a given time point (p) for a given mouse (m)
                qtpm = q(t,rT,deltaT,b0,q0,deltaD) #similarly for qtpm: ceqs for a particular test, time point, mouse     
                bTlist[p][m]=btpm #2D list of CFUs, organized by time point and mouse
                qTlist[p][m]=qtpm #2D list of CEQs
    
        #Best fit line using final two points
        sumqtT = 0. #accumulators for finding averages
        sumtT = 0.
        sumt2T = 0.
        sumqT = 0.
        sumq2T = 0.
        for p in range(1,3): #double loop over points... note that range(1,3) is equivalent to the list [1, 2]
                                                        # i.e., this loop is over the final two time points.
            t = tList[p]
            for m in range(nMice): #...and mice
                lnq = log(qTlist[p][m])
                sumqtT += lnq*t
                sumtT += t
                sumt2T += t * t
                sumqT += lnq
                sumq2T += lnq * lnq
        dtdtT = 0.
        avqtT = sumqtT / 2. / nMice # <ln Q * t>
        avqT = sumqT / 2. / nMice # <ln Q>
        avtT = sumtT / 2. / nMice # <t>
        avt2T = sumt2T / 2. / nMice # <t^2>
        avq2T = sumq2T / 2. / nMice # <(ln q)^2>
        slope = (avqtT - avtT*avqT)/(avt2T - avtT * avtT) #covariance(Q,t)/variance(t)
        intercept = avqT - slope * avtT

        #calculate ssr
        varqT = (avq2T - avqT * avqT)
        ssrDeltaD0 = varqT * nMice * 2.
        ssr = 0.
        for p in range(1,3):
            qFit = slope * tList[p] + intercept
            for m in range(nMice):
                ssr += (log(qTlist[p][m]) - qFit)**2
        
        #f test for nested models
        f = 2 * nMice * (ssrDeltaD0 - ssr) / ssr
        p = 1 - stats.f.cdf(f, 1, 2 * nMice - 2)
        pList.append(p)
        if( p < alpha ):
            nSignificant += 1
        averageP += p
        averageF += f
                
    averageP = averageP / nTests
    averageF = averageF / nTests
    return averageF, averageP, nSignificant/nTests


In [6]:
#Simulation parameters

rT = 0.079 #replication rate
deltaT = 0.28 #death rate
deltaD = 0.036 #decay rate of CEQs of dead bacteria

#How many simulations
nTests = 10000
#Threshold for statistical significance
alpha = 0.05

#Time between final two time points
weeksApart = 4
#mice per group (multiply by 3 to get no. of mice for the whole experiment)
nMice = 12

#output header
print( "mean F statistic   mean P-value \tstatistical power" )

#Perform the analysis
averageF, averageP, powerNmWa = power(nTests, alpha, nMice, weeksApart, [rT,deltaT,deltaD] )

#output results
print( averageF, averageP, powerNmWa )


mean F statistic   mean P-value 	statistical power
23.11729861154807 0.01774163265577969 0.9269
